## 손실 함수

### 오차제곱합

In [10]:
# 소수점 표현이 교재와 달라 format 함수로 자릿수를 맞춰줌
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# print(sum_squares_error(np.array(y), np.array(t)))
print(format(sum_squares_error(np.array(y), np.array(t)), '.18f'))

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# print(sum_squares_error(np.array(y), np.array(t)))
print(format(sum_squares_error(np.array(y), np.array(t)), '.17f'))


0.097500000000000031
0.59750000000000003


### 교차손실함수

In [42]:
# np.log()에 0을 입력하게 되어 오류가 발생

def cross_entropy_error(y, t):
    return -np.sum(t * np.log(y))

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

/var/folders/hn/nrs0d2b91f1f7jdk9wdd8yfc0000gn/T/ipykernel_3926/1968273501.py:4: RuntimeWarning: divide by zero encountered in log
  return -np.sum(t * np.log(y))
/var/folders/hn/nrs0d2b91f1f7jdk9wdd8yfc0000gn/T/ipykernel_3926/1968273501.py:4: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(t * np.log(y))


nan

In [43]:
# 위 에러를 방지하기 위해 아주 작은 값, delta를 추가
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

### 미니배치 학습

In [60]:
import numpy as np
from mnist import load_mnist

# one_hot_label=False로 설정하면 t_train, t_test의 값이 변함
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000,)


In [52]:
np.random.choice(60000, 10)

array([38624, 23946,  3365,  5518, 43332, 30918, 43674, 19923, 11125,
       33168])

In [49]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

### 배치용 교차 엔트로피 오차 구현하기

In [53]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    # t가 one-hot이므로 0과 곱해지는 부분은 무시가 됨
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [54]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    # t로 나타낸 인덱스를 y에서 추출해야 함 
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

## 수치 미분